In [ ]:
#! /usr/bin/env python

###################################################################################
# Copyright   2015, Pittsburgh Supercomputing Center (PSC).  All Rights Reserved. #
# =============================================================================== #
#                                                                                 #
# Permission to use, copy, and modify this software and its documentation without #
# fee for personal use within your organization is hereby granted, provided that  #
# the above copyright notice is preserved in all copies and that the copyright    #
# and this permission notice appear in supporting documentation.  All other       #
# restrictions and obligations are defined in the GNU Affero General Public       #
# License v3 (AGPL-3.0) located at http://www.gnu.org/licenses/agpl-3.0.html  A   #
# copy of the license is also provided in the top level of the source directory,  #
# in the file LICENSE.txt.                                                        #
#                                                                                 #
###################################################################################

import logging

import os.path
from phacsl.utils.collections.phacollections import SingletonMetaClass
import pyrheautils
import random
from collections import deque, defaultdict
from policybase import TransferDestinationPolicy as BaseTransferDestinationPolicy
from facilitybase import CareTier, tierToQueueMap
from map_transfer_matrix import parseFacilityData
import schemautils
import yaml

_validator = None
_constants_values = '$(MODELDIR)/constants/transferbydrawwithreplacement_constants.yaml'
_constants_schema = 'transferbydrawwithreplacement_constants_schema.yaml'
_constants = None


schemautils.setSchemaBasePath('/home/welling/git/pyRHEA_github/src/schemata')
    
pyrheautils.PATH_STRING_MAP['MODELDIR'] = '/home/welling/git/pyRHEA_github/models/OrangeCounty2013'
_constants = pyrheautils.importConstants(_constants_values,
                                         _constants_schema)
_constants['transferFilePaths']=['$(MODELDIR)/direct_transfer_counts.yaml']

print _constants
nmDict = CareTier.names
#tierFacSets = {tier: set(self.getTierAddrMap(tier).keys()) for tier in nmDict.keys()}
tbl = {}
pairsSeen = set()
for transferMatrixFilePath in _constants['transferFilePaths']:
    print 'Importing the weight data file %s' % transferMatrixFilePath
    rawTbl = pyrheautils.importConstants(transferMatrixFilePath,
                                         _constants['transferFileSchema'])
    for srcName, rec in rawTbl.items():
        if srcName not in tbl:
            tbl[srcName] = {}
        for destName, ct in rec.items():
            if (srcName, destName) in pairsSeen:
                raise RuntimeError('Duplicate weight table entries for %s -> %s' %
                                    (srcName, destName))
            else:
                pairsSeen.add((srcName, destName))
                tbl[srcName][destName] = float(ct)
facDict = parseFacilityData(pyrheautils.pathTranslate('$(MODELDIR)/facilityfactsCurrent2013'))
#print 'IMPLEMENTING SPECIAL PATCH FOR WAUK_2615_H'
#facDict['WAUK_2615_H'] = {'category':'HOSPITAL'}
#print 'FIND OUT THE REAL ANSWER AND DELETE THIS!'
facDict.update(parseFacilityData(pyrheautils.pathTranslate('$(MODELDIR)/facilityfacts_excluded')))


totCt = 0.0
for src, rec in tbl.items():
    for dst, ct in rec.items():
        totCt += float(ct)
print 'totCt is %s' % totCt
#for key, val in catCtD.items():
#    print 'category %s: %s' % (key, val)
allSrcTbl = defaultdict(lambda: 0)
catCtD = defaultdict(lambda: 0)
srcCatCtD = defaultdict(lambda: defaultdict(lambda: 0)) # a defaultdict of defaultdicts
for srcName in tbl.keys():
    for destName, ct in tbl[srcName].items():
        allSrcTbl[destName] += ct
        destCategory = facDict[destName]['category']
        catCtD[destCategory] += float(ct)
        srcCatCtD[srcName][destCategory] += float(ct)
totCt = sum(allSrcTbl.values())
print 'totCt after gather: %s' % totCt
for destName, ct in allSrcTbl.items():
    allSrcTbl[destName] /= totCt

        
#        for tier in nmDict.keys():
#            if tier not in tbl[srcName]:
#                tbl[srcName][tier] = []
#                totTbl[srcName][tier] = 0.0
#            wtL = tbl[srcName][tier]
#            wtSum = totTbl[srcName][tier]
#            for destName, ct in rec.items():
#                if destName in tierFacSets[tier]:
#                    wtL.append((ct, (destName, self.getTierAddrMap(tier)[destName])))
#                    wtSum += ct
#            totTbl[srcName][tier] = wtSum
print 'Import complete.'


In [ ]:
print 'totals check: %s vs %s' % (sum(catCtD.values()), totCt)
print 'Fractions:'
for key, val in catCtD.items():
    print '  %s: %s' % (key, (val/totCt))

In [ ]:
srcL = tbl.keys()[:]
srcL.sort()
oD = {}
oD['COMMUNITY'] = {key:val for key,val in allSrcTbl.items()}  # get rid of defaultdict
with open('com_to_fac_marginalized_direct.yaml', 'w') as f:
    yaml.safe_dump(oD, f, default_flow_style=True, indent=4,
                    encoding='utf-8', width=130, explicit_start=True)


In [ ]:
print srcCatCtD['LPAL']

In [ ]:
newSrcCatCtD = {}
for src, catD in srcCatCtD.items():
    tot = sum(catD.values())
    newCatCtD = {cat: ct/tot for cat, ct in catD.items()}
    newSrcCatCtD[src] = newCatCtD
srcCatCtD = newSrcCatCtD

In [ ]:
print srcCatCtD['LPAL']

In [ ]:
with open('fac_to_cat_marginalized_direct.yaml', 'w') as f:
    yaml.safe_dump(srcCatCtD, f, default_flow_style=True, indent=4,
                    encoding='utf-8', width=130, explicit_start=True)